In [1]:
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
import pandas as pd
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict
import time


In [2]:
# download data
raw_data = pd.read_csv('heart.csv')

print(raw_data)
print(raw_data.columns[:9])
print(raw_data.loc[:,raw_data.columns[:9]])

     sbp  tobacco    ldl  adiposity  famhist  typea  obesity  alcohol  age  \
0    160    12.00   5.73      23.11  Present     49    25.30    97.20   52   
1    144     0.01   4.41      28.61   Absent     55    28.87     2.06   63   
2    118     0.08   3.48      32.28  Present     52    29.14     3.81   46   
3    170     7.50   6.41      38.03  Present     51    31.99    24.26   58   
4    134    13.60   3.50      27.78  Present     60    25.99    57.34   49   
5    132     6.20   6.47      36.21  Present     62    30.77    14.14   45   
6    142     4.05   3.38      16.20   Absent     59    20.81     2.62   38   
7    114     4.08   4.59      14.60  Present     62    23.11     6.72   58   
8    114     0.00   3.83      19.40  Present     49    24.86     2.49   29   
9    132     0.00   5.80      30.96  Present     69    30.11     0.00   53   
10   206     6.00   2.95      32.27   Absent     72    26.81    56.06   60   
11   134    14.10   4.44      22.39  Present     65    23.09    

In [3]:
label = raw_data['chd']
# data = raw_data.loc[:,raw_data.columns[:9]].replace(['Present', 'Absent'],[0, 1])
# print(data)
# data = data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)), axis=0)
# all_data = data
# all_data['chd'] = label
# all_data.describe()
# print(all_data)

data = raw_data.loc[:,raw_data.columns[:9]].replace(['Present', 'Absent'],[0, 1])
# 归一化data
data = data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)), axis=0)

all_data = data
all_data['chd'] = label
all_data.describe()

# print(all_data)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
count,4.620000e+02,4.620000e+02,4.620000e+02,4.620000e+02,4.620000e+02,4.620000e+02,4.620000e+02,4.620000e+02,4.620000e+02,462.000000
mean,-2.691450e-17,-4.205390e-18,4.998407e-17,4.277483e-17,4.998407e-17,-1.826341e-17,8.026288e-17,3.172066e-17,9.612321e-19,0.346320
std,1.751822e-01,1.472123e-01,1.443142e-01,2.176419e-01,4.933567e-01,1.510390e-01,1.321732e-01,1.663228e-01,2.981420e-01,0.476313
min,-3.190328e-01,-1.165272e-01,-2.620435e-01,-5.221463e-01,-5.844156e-01,-6.169830e-01,-3.558379e-01,-1.157986e-01,-5.676738e-01,0.000000
25%,-1.224516e-01,-1.148445e-01,-1.015906e-01,-1.575310e-01,-5.844156e-01,-9.390609e-02,-9.595711e-02,-1.123337e-01,-2.411432e-01,0.000000
50%,-3.698154e-02,-5.242466e-02,-2.789719e-02,1.981170e-02,4.155844e-01,-1.598402e-03,-7.500394e-03,-6.477610e-02,4.457108e-02,0.000000
75%,8.267658e-02,5.975483e-02,7.314811e-02,1.628187e-01,4.155844e-01,1.060939e-01,7.695695e-02,4.652562e-02,2.486527e-01,1.000000
max,6.809672e-01,8.834728e-01,7.379565e-01,4.778537e-01,4.155844e-01,3.830170e-01,6.441621e-01,8.842014e-01,4.323262e-01,1.000000


In [4]:
#重组数据集，保证label数量相等
one_label_result = all_data[(all_data.chd == 1)]
zero_label_result = all_data[(all_data.chd == 0)]

one_label_length = len(one_label_result)
zero_label_length = len(zero_label_result)

small_len = one_label_length if one_label_length < zero_label_length else zero_label_length;

one_index = random.sample(list(one_label_result.index.values), small_len)
zero_index = random.sample(list(zero_label_result.index.values), small_len)

new_data = pd.concat([one_label_result.ix[one_index], zero_label_result.ix[zero_index]])
new_data.describe()
print(new_data)
# print(one_label_length)
# print(zero_label_length)
# print(small_len)
print(one_index)

          sbp   tobacco       ldl  adiposity   famhist     typea   obesity  \
27   0.057036  0.175139  0.034821   0.059952  0.415584  0.090709 -0.159477   
11  -0.036982  0.335396 -0.020929  -0.084384 -0.584416  0.183017 -0.092664   
415 -0.036982 -0.081271 -0.083646  -0.139769 -0.584416  0.075325 -0.047180   
382  0.014301 -0.062681  0.464786   0.115616 -0.584416  0.321479  0.147613   
106 -0.259204 -0.068450 -0.028594  -0.011657  0.415584  0.198402 -0.117758   
215  0.185241  0.132511  0.232033   0.262749  0.415584  0.167632  0.160160   
228 -0.002794 -0.052425  0.025761   0.167644 -0.584416 -0.063137  0.037826   
18   0.168147 -0.033194  0.189524   0.242329 -0.584416  0.121479  0.102129   
17   0.065583  0.220011  0.247364   0.278413 -0.584416  0.383017  0.209720   
390 -0.071170 -0.099220 -0.077375  -0.094454 -0.584416  0.244555 -0.053454   
113  0.304899 -0.116527  0.259211   0.271140 -0.584416 -0.278521 -0.024282   
32  -0.139546  0.095011  0.058514   0.294917 -0.584416 -0.001598

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


In [5]:
# 数据分10份，拿一份做测试集，九份做训练集
train_data_size = int(small_len * 2 * 0.9)
test_data_size = int(small_len * 2 * 0.1)

train_data_index = random.sample(list(new_data.index.values), train_data_size)
train_data = new_data.ix[train_data_index]

test_data_index = list(set(new_data.index.values).difference(set(train_data_index)))
test_data = new_data.ix[test_data_index]

train_label = train_data['chd']
train_data = train_data.loc[:,raw_data.columns[:9]]

test_label = test_data['chd']
test_data = test_data.loc[:,raw_data.columns[:9]]
# print(train_data.shape, train_label.shape)
# print(test_data.shape, test_label.shape)
print(train_data)

          sbp   tobacco       ldl  adiposity   famhist     typea   obesity  \
79  -0.259204 -0.103707  0.081510  -0.069559 -0.584416  0.059940 -0.010167   
7   -0.207922  0.014242 -0.010476  -0.302286 -0.584416  0.136863 -0.092036   
12  -0.173734 -0.116527 -0.199326  -0.429559  0.415584  0.090709 -0.140342   
220 -0.002794 -0.116527 -0.111521  -0.375013  0.415584  0.013786 -0.180807   
454  0.065583 -0.096014  0.005552   0.073098  0.415584  0.106094  0.064802   
365 -0.054076 -0.093450 -0.025807  -0.164104  0.415584 -0.078521  0.002067   
322  0.236523  0.075780 -0.119883   0.108903  0.415584 -0.278521 -0.052199   
237  0.321993  0.068088  0.010430   0.019392 -0.584416 -0.109291  0.039394   
426  0.031395 -0.074220  0.201371   0.127924 -0.584416  0.059940  0.160473   
352  0.321993 -0.078066  0.246667   0.300791 -0.584416 -0.170829  0.055392   
303 -0.088264 -0.116527  0.255727   0.095476 -0.584416  0.106094  0.025592   
22   0.099771 -0.106912  0.114263   0.240091 -0.584416  0.136863

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [6]:
# Define paramaters for the model
# learning_rate = 0.01
# batch_size = 16
# n_epochs = 1000

In [7]:
# X = tf.placeholder(dtype = np.float32, shape = [batch_size, 9], name='X')
# Y = tf.placeholder(dtype = np.float32, shape = [batch_size, 2], name='Y')

# W1 = tf.Variable(tf.random_normal([9, 20]), name='W1')
# b1 = tf.Variable(tf.random_normal([1, 20]), name='b1')

# W2 = tf.Variable(tf.random_normal([20, 10]), name='W2')
# b2 = tf.Variable(tf.random_normal([1, 10]), name='b2')

# W3 = tf.Variable(tf.random_normal([10, 2]), name='W3')
# b3 = tf.Variable(tf.random_normal([1, 2]), name='b3')

# Z1 = tf.matmul(X, W1) + b1
# A1 = tf.nn.relu(Z1)
# Z2 = tf.matmul(A1, W2) + b2
# A2 = tf.nn.relu(Z2)
# logits = tf.matmul(A2, W3) + b3

# entropy = tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = logits)

# loss = tf.reduce_mean(entropy)

# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# def to_one_hotting(labels):
#     return (np.arange(2) == labels[:,None]).astype(np.float32)

# with tf.Session() as sess:
#     start_time = time.time()
#     sess.run(tf.global_variables_initializer())	
#     n_batches = int(len(train_data)/batch_size)
#     for i in range(n_epochs): 
#         total_loss = 0

#         for index in range(n_batches):
#             X_batch = train_data[index*batch_size:(index+1)*batch_size].values
#             Y_batch = train_label[index*batch_size:(index+1)*batch_size].values
#             Y_batch = to_one_hotting(Y_batch)
#             _, loss_batch, get_entropy, get_logits = sess.run([optimizer, loss, entropy, logits], feed_dict={X: X_batch, Y: Y_batch})
#             total_loss += loss_batch
            
#         if i%100 == 0:
#             print('Average loss epoch :{0}'.format(total_loss/n_batches))

#     print('Total time: {0} seconds'.format(time.time() - start_time))

#     print('Optimization Finished!')

#     # test the model
#     n_batches = int(len(test_data)/batch_size)
#     total_correct_preds = 0
#     for index in range(n_batches):
#         X_batch = test_data[index*batch_size:(index+1)*batch_size].values
#         Y_batch = test_label[index*batch_size:(index+1)*batch_size].values
#         Y_batch = to_one_hotting(Y_batch)
#         _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], feed_dict={X: X_batch, Y:Y_batch}) 
#         preds = tf.nn.softmax(logits_batch)
#         correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
#         accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
#         total_correct_preds += sess.run(accuracy)

#     print('Accuracy:',format(total_correct_preds/len(test_data)))

In [8]:
# GRADED FUNCTION: create_placeholders

def create_placeholders(n_x, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, size of an image vector (num_px * num_px = 64 * 64 * 3 = 12288)
    n_y -- scalar, number of classes (from 0 to 5, so -> 6)
    
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [n_y, None] and dtype "float"
    
    Tips:
    - You will use None because it let's us be flexible on the number of examples you will for the placeholders.
      In fact, the number of examples during test/train is different.
    """

    ### START CODE HERE ### (approx. 2 lines)
    X = tf.placeholder(shape=[n_x, None], dtype=tf.float32)
    Y = tf.placeholder(shape=[n_y, None], dtype=tf.float32)
    ### END CODE HERE ###
    
    return X, Y

In [9]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters():
    """
    Initializes parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [9, 20]
                        b1 : [1, 20]
                        W2 : [20, 10]
                        b2 : [1, 10]
                        W3 : [10, 2]
                        b3 : [1 2]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2, W3, b3
    """
    
#     tf.set_random_seed(1)                   # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 6 lines of code)
#     W1 = tf.get_variable("W1", [20, 288], initializer = tf.contrib.layers.xavier_initializer())
#     b1 = tf.get_variable("b1", [20,1], initializer = tf.contrib.layers.xavier_initializer())
#     W2 = tf.get_variable("W2", [10,20], initializer = tf.contrib.layers.xavier_initializer())
#     b2 = tf.get_variable("b2", [10,1], initializer = tf.contrib.layers.xavier_initializer())
#     W3 = tf.get_variable("W3", [2,10], initializer = tf.contrib.layers.xavier_initializer())
#     b3 = tf.get_variable("b3", [2,1], initializer = tf.contrib.layers.xavier_initializer())
    
    W1 = tf.Variable(tf.random_normal([20, 9]), name='W1')
    b1 = tf.Variable(tf.random_normal([20, 1]), name='b1')

    W2 = tf.Variable(tf.random_normal([10, 20]), name='W2')
    b2 = tf.Variable(tf.random_normal([10, 1]), name='b2')

    W3 = tf.Variable(tf.random_normal([1, 10]), name='W3')
    b3 = tf.Variable(tf.random_normal([1, 1]), name='b3')
    ### END CODE HERE ###

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [10]:
tf.reset_default_graph()
with tf.Session() as sess:
    parameters = initialize_parameters()
    print("W1 = " + str(parameters["W1"]))
    print("b1 = " + str(parameters["b1"]))
    print("W2 = " + str(parameters["W2"]))
    print("b2 = " + str(parameters["b2"]))

W1 = <tf.Variable 'W1:0' shape=(20, 9) dtype=float32_ref>
b1 = <tf.Variable 'b1:0' shape=(20, 1) dtype=float32_ref>
W2 = <tf.Variable 'W2:0' shape=(10, 20) dtype=float32_ref>
b2 = <tf.Variable 'b2:0' shape=(10, 1) dtype=float32_ref>


In [11]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    ### START CODE HERE ### (approx. 5 lines)              # Numpy Equivalents:
    Z1 = tf.add(tf.matmul(W1, X), b1)                                            # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                              # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                                              # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                              # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)                                              # Z3 = np.dot(W3,Z2) + b3
    ### END CODE HERE ###
    
    return Z3

In [12]:
tf.reset_default_graph()
# print(train_data.shape)
# print(train_label.shape[0])

with tf.Session() as sess:
    X, Y = create_placeholders(288, 288)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    print("Z3 = " + str(Z3))

ValueError: Dimensions must be equal, but are 9 and 288 for 'MatMul' (op: 'MatMul') with input shapes: [20,9], [288,?].

In [13]:
# GRADED FUNCTION: compute_cost 

def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    ### START CODE HERE ### (1 line of code)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = labels))
    ### END CODE HERE ###
    
    return cost

In [14]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholders(288, 288)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    print("cost = " + str(cost))

ValueError: Dimensions must be equal, but are 9 and 288 for 'MatMul' (op: 'MatMul') with input shapes: [20,9], [288,?].

In [15]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    Arguments:
    X_train -- training set, of shape (input size = 12288, number of training examples = 1080)
    Y_train -- test set, of shape (output size = 6, number of training examples = 1080)
    X_test -- training set, of shape (input size = 12288, number of training examples = 120)
    Y_test -- test set, of shape (output size = 6, number of test examples = 120)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
#     ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []       # To keep track of the cost
    
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)

    X = tf.placeholder(dtype = np.float32, shape = [9, None])
    Y = tf.placeholder(dtype = np.float32, shape = [1, None])    
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameters()
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3 = forward_propagation(X, parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z3, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            print(minibatch_size)
            print(m)
            print(m / minibatch_size)
        
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size)
            
            
            
            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                ### END CODE HERE ###
                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters

In [17]:
parameters = model(train_data, train_label, test_data, test_label)


32
9
0.28125


TypeError: unhashable type: 'slice'